In [1]:
# !pip install map_boxes
# !pip install tqdm
# !pip install pycocotools

In [2]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [3]:
GT_JSON = './detection_backup/dataset/train.json'
PRED_CSV = './submission_latest.csv'

# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

In [4]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2 confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''

new_pred = []

file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

'''
TODO create new_pred
'''
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f'{file_names[i]} empty box')

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(' '))
    
    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

4883it [00:03, 1545.23it/s]


In [5]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2, confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
gt = []
    
   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
'''
coco = COCO(GT_JSON)

'''
TODO create gt
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!


In [6]:
num = 0
for line in gt:
    print(line)
    num += 1
    if num >= 10:
        break

['train/0000.jpg', 0, 197.6, 745.4, 193.7, 663.4]
['train/0001.jpg', 3, 0.0, 57.6, 407.4, 588.0]
['train/0001.jpg', 7, 0.0, 144.6, 455.6, 637.2]
['train/0001.jpg', 4, 722.3, 996.5999999999999, 313.4, 565.3]
['train/0001.jpg', 5, 353.2, 586.9, 671.0, 774.4]
['train/0001.jpg', 5, 3.7, 781.9000000000001, 448.5, 690.5]
['train/0001.jpg', 0, 425.3, 641.7, 681.9, 861.7]
['train/0001.jpg', 7, 92.4, 231.6, 601.7, 654.8000000000001]
['train/0001.jpg', 0, 622.4, 695.1999999999999, 686.5, 780.7]
['train/0002.jpg', 3, 267.9, 899.5, 165.2, 678.2]


In [10]:
num = 0
print(len(new_pred))
for line in new_pred:
    print(line)
    num += 1
    if num >= 30:
        break

488300
['train/0000.jpg', '8', '0.10807686', 224.83769, 240.25734, 512.94183, 587.07587]
['train/0000.jpg', '8', '0.108024135', 233.01276, 248.53638, 513.02966, 587.18176]
['train/0000.jpg', '8', '0.10798689', 227.17767, 282.98053, 246.45956, 419.197]
['train/0000.jpg', '8', '0.107746646', 273.6042, 329.43738, 279.79297, 453.17218]
['train/0000.jpg', '8', '0.10760053', 255.16011, 270.89148, 503.88446, 579.74835]
['train/0000.jpg', '8', '0.10756916', 368.22604, 384.1832, 527.4729, 603.26465]
['train/0000.jpg', '8', '0.107542284', 431.5567, 447.31384, 573.6112, 649.9646]
['train/0000.jpg', '8', '0.10750076', 440.26657, 455.83853, 581.74396, 658.79803]
['train/0000.jpg', '8', '0.107491575', 97.862175, 152.98077, 423.45016, 594.316]
['train/0000.jpg', '8', '0.10742503', 294.9772, 310.51804, 503.92453, 580.4899]
['train/0000.jpg', '8', '0.107387014', 114.79584, 169.43515, 248.08539, 420.57758]
['train/0000.jpg', '8', '0.10736487', 163.41309, 218.61249, 374.8345, 544.1007]
['train/0000.jpg',

In [11]:
'''
calculate mAP
'''

'''
Special mission you sould create mean_average_precision_for_boxes function, not library
'''

mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.000000 |    3966
1                              | 0.000000 |    6352
2                              | 0.000000 |     897
3                              | 0.000000 |     936
4                              | 0.000000 |     982
5                              | 0.000000 |    2943
6                              | 0.000000 |    1263
7                              | 0.000000 |    5178
8                              | 0.000000 |     159
9                              | 0.000000 |     468
mAP: 0.000000
0.0
